In [1]:
import os
os.chdir("../")

In [2]:
#update the entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path

In [5]:
#update the configuration manager in src config

from src.mlproject.constants import *
from src.mlproject.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAM_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
        )
        
        return data_transformation_config

In [6]:
from src.mlproject import logger
from sklearn.model_selection import train_test_split
import pandas as pd

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        
    ## Note: we can add different data transformation techniques such as scaler, PCA and all
    # we can perform all kinds of EDA in ML cycle here before passing this data to the model
    
    def train_test_spliting(self):
        data = pd.read_csv(self.config.data_path)
        
        #split the data into training and test sets (0.75, 0.25) split.
        train, test = train_test_split(data)
        
        train.to_csv(os.path.join(self.config.root_dir, "train.csv"),index = False)
        test.to_csv(os.path.join(self.config.root_dir, "test.csv"),index = False)
        
        logger.info("Splitted data into training and test sets")
        logger.info(train.shape)
        logger.info(test.shape)
        
        print(train.shape)
        print(test.shape)

In [7]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.train_test_spliting()
except Exception as e:
    raise e

[2024-03-19 18:33:51,823: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-03-19 18:33:51,837: INFO: common: yaml file: params.yaml loaded successfully]


[2024-03-19 18:33:51,843: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-03-19 18:33:51,852: INFO: common: created directory at: artifacts]
[2024-03-19 18:33:51,854: INFO: common: created directory at: artifacts/data_transformation]
[2024-03-19 18:33:51,941: INFO: 3850296996: Splitted data into training and test sets]
[2024-03-19 18:33:51,942: INFO: 3850296996: (1199, 12)]
[2024-03-19 18:33:51,943: INFO: 3850296996: (400, 12)]
(1199, 12)
(400, 12)
